<a href="https://colab.research.google.com/github/nomomon/drl-js/blob/main/tic-tac-toe/notebookipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np

# Model

In [68]:
policy = tf.keras.Sequential()

policy.add(tf.keras.layers.Dense(100, activation=tf.nn.relu, input_shape = (9, )))
policy.add(tf.keras.layers.Dropout(0.1))
policy.add(tf.keras.layers.Dense(100, activation=tf.nn.relu))
policy.add(tf.keras.layers.Dropout(0.1))
policy.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))


policy.compile(
    optimizer = tf.keras.optimizers.Adam(), 
    loss = tf.keras.losses.BinaryCrossentropy(), 
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

In [69]:
policy.predict([[1, 0, -1, 0, -1, 0, 1, 0, 0]])

array([[0.484995]], dtype=float32)

In [70]:
policy.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 100)               1000      
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_11 (Dense)            (None, 100)               10100     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_12 (Dense)            (None, 1)                 101       
                                                                 
Total params: 11,201
Trainable params: 11,201
Non-trainable params: 0
_________________________________________________________________


In [71]:
# 0 1 2
# 3 4 5
# 6 7 8

def gameState(board):
    lines = [
        [0, 1, 2],
        [3, 4, 5],
        [6, 7, 8],
        [0, 3, 6],
        [1, 4, 7],
        [2, 5, 8],
        [0, 4, 8],
        [2, 4, 6],
    ]
    for line in lines:
        if (board[line[0]] == board[line[1]] and board[line[1]] == board[line[2]] and board[line[1]] != 0):
            return board[line[1]]
    return 0

In [72]:
gameState([0, 0, 0, 
           1, 1, 1, 
           1, 0, 0])

1

In [73]:
def chooseAction(policy, board):
    probs = [0, 0, 0,
             0, 0, 0, 
             0, 0, 0]

    for i, cell in enumerate(board):
        if cell == 0:
            playBoard = board
            playBoard[i] = 1

            probs[i] = policy.predict([playBoard])[0][0]

    maxprob = probs[0]
    maxi = 0

    for i, prob in enumerate(probs):
        if prob >= maxprob:
            maxprob = prob
            maxi = i

    return maxi

In [74]:
chooseAction(policy, [0, 0, 1,
                      1, 1, 0,
                      0, 0, 1])

6

In [75]:
def getData(policy):
    X_all = []
    y_all = []
    
    X = []
    y = []

    board = [0, 0, 0,
             0, 0, 0,
             0, 0, 0]

    winner = 0

    for i in range(9):
        action = chooseAction(policy, board.copy())
        
        board[action] = 1


        X.append(board)
        y.append((i % 2) * 2 - 1)

        if(gameState(board) != 0):
            winner = (i % 2) * 2 - 1
            break

        board = (np.array(board) * -1).tolist()

    y = list(map(lambda q: ((q == winner) - 0), y))
    
    X_all.extend(X)
    y_all.extend(y)

    X = []
    y = []

    board = [0, 0, 0,
             0, 0, 0,
             0, 0, 0]

    winner = 0

    for i in range(9):
        if(i % 2 == 1):
            action = chooseAction(policy, board.copy())
        else:
            action = np.random.randint(0, 9, size = 1)[0]
            while(board[action] != 0):
                action = np.random.randint(0, 9, size = 1)[0]

        board[action] = 1


        X.append(board)
        y.append((i % 2) * 2 - 1)

        if(gameState(board) != 0):
            winner = (i % 2) * 2 - 1
            break

        board = (np.array(board) * -1).tolist()
    
    y = list(map(lambda q: ((q == winner) - 0), y))

    X_all.extend(X)
    y_all.extend(y)

    X = []
    y = []

    board = [0, 0, 0,
             0, 0, 0,
             0, 0, 0]

    winner = 0

    for i in range(9):
        if(i % 2 == 0):
            action = chooseAction(policy, board.copy())
        else:
            action = np.random.randint(0, 9, size = 1)[0]
            while(board[action] != 0):
                action = np.random.randint(0, 9, size = 1)[0]

        board[action] = 1


        X.append(board)
        y.append((i % 2) * 2 - 1)

        if(gameState(board) != 0):
            winner = (i % 2) * 2 - 1
            break

        board = (np.array(board) * -1).tolist()
    
    y = list(map(lambda q: ((q == winner) - 0), y))

    X_all.extend(X)
    y_all.extend(y)

    return X_all, y_all

In [76]:
getData(policy)

([[0, 0, 0, 0, 0, 0, 1, 0, 0],
  [0, 0, 0, 0, 0, 1, -1, 0, 0],
  [0, 1, 0, 0, 0, -1, 1, 0, 0],
  [0, -1, 1, 0, 0, 1, -1, 0, 0],
  [0, 1, -1, 0, 0, -1, 1, 1, 0],
  [1, -1, 1, 0, 0, 1, -1, -1, 0],
  [-1, 1, -1, 0, 1, -1, 1, 1, 0],
  [0, 0, 0, 0, 0, 1, 0, 0, 0],
  [0, 0, 0, 0, 0, -1, 0, 1, 0],
  [0, 0, 1, 0, 0, 1, 0, -1, 0],
  [0, 0, -1, 0, 0, -1, 1, 1, 0],
  [1, 0, 1, 0, 0, 1, -1, -1, 0],
  [-1, 0, -1, 0, 1, -1, 1, 1, 0],
  [1, 0, 1, 0, -1, 1, -1, -1, 1],
  [0, 0, 0, 0, 0, 0, 1, 0, 0],
  [1, 0, 0, 0, 0, 0, -1, 0, 0],
  [-1, 0, 0, 0, 0, 1, 1, 0, 0],
  [1, 0, 0, 0, 1, -1, -1, 0, 0],
  [-1, 1, 0, 0, -1, 1, 1, 0, 0],
  [1, -1, 0, 1, 1, -1, -1, 0, 0],
  [-1, 1, 1, -1, -1, 1, 1, 0, 0],
  [1, -1, -1, 1, 1, -1, -1, 0, 1]],
 [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])

In [ ]:
for i in range(100):
    X, y = getData(policy)
    hist = policy.fit(
        X, y, 
        epochs = 1, 
        verbose = 0
    ).history

    print("epoch {:2} | loss: {:1.5} - acc: {:2.2%}".format(i, hist["loss"][0], hist["binary_accuracy"][0]))

epoch  0 | loss: 0.69269 - acc: 54.55%
epoch  1 | loss: 0.69262 - acc: 50.00%
epoch  2 | loss: 0.74752 - acc: 45.45%
epoch  3 | loss: 0.66556 - acc: 63.64%
epoch  4 | loss: 0.70568 - acc: 60.00%
epoch  5 | loss: 0.69205 - acc: 78.26%
epoch  6 | loss: 0.68798 - acc: 59.09%
epoch  7 | loss: 0.61578 - acc: 81.82%
epoch  8 | loss: 0.61488 - acc: 86.96%
epoch  9 | loss: 0.59546 - acc: 90.00%
epoch 10 | loss: 0.68808 - acc: 60.87%
epoch 11 | loss: 0.75037 - acc: 33.33%
epoch 12 | loss: 0.7422 - acc: 50.00%
epoch 13 | loss: 0.54999 - acc: 100.00%
epoch 14 | loss: 0.70969 - acc: 43.48%
epoch 15 | loss: 0.69103 - acc: 45.00%
epoch 16 | loss: 0.83233 - acc: 18.18%
epoch 17 | loss: 0.67883 - acc: 70.59%
epoch 18 | loss: 0.59919 - acc: 86.36%
epoch 19 | loss: 0.67987 - acc: 69.57%
epoch 20 | loss: 0.61276 - acc: 71.43%
epoch 21 | loss: 0.67557 - acc: 47.62%
epoch 22 | loss: 0.70011 - acc: 50.00%
epoch 23 | loss: 0.61429 - acc: 85.00%
epoch 24 | loss: 0.62802 - acc: 64.00%
epoch 25 | loss: 0.81255 

# Web demo

Play against the AI

In [54]:
from IPython.display import clear_output 

def symbol(x):
    if x == 1:
        return "X"
    elif x == -1:
        return "O"
    else:
        return "?"

def printBoard(board):
    clear_output()
    cBoard = list(map(symbol, board))
    for i in range(0, 3):
        row = ""
        for j in range(0, 3):
            row += (cBoard[j + i * 3]) if (cBoard[j + i * 3] != "?") else str(j + i * 3 + 1)
            if j != 2:
                row += " | "
        print(row)
        if i != 2:
            print("---------")

def play(policy):
    player = (int(input("which player you want to be? (1 or 2) ")) + 1) % 2 

    board = [0, 0, 0,
             0, 0, 0,
             0, 0, 0]

    winner = 0

    for i in range(9):
        if (i % 2 == player):
            printBoard(board)
            action = int(input("what cell? ")) - 1
        else:
            action = chooseAction(policy, board.copy())

        board[action] = 1

        if(gameState(board) != 0):
            winner = i % 2 == player
            break

        board = (np.array(board) * -1).tolist()
    
    printBoard(board)
    if(gameState(board) != 0):
        print("\nwinner is the", "humen" if winner else "ai")
    else:
        print("\nit's a tie!")

In [57]:
play(policy)

O | O | 3
---------
4 | O | 6
---------
X | X | X

winner is the humen


# Deploy to TF.js

In [62]:
 pip install tensorflowjs[wizard]

In [ ]:
pip install -U ipython

In [ ]:
policy.save("./model/")

In [ ]:
!tensorflowjs_converter --input_format=keras_saved_model /content/model /content/tfjs_model